In [37]:
# Import required libraries.
import hvplot.pandas
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from collections import Counter
import sqlalchemy

## Pull in Tweet data from CSV and use Counter to count tweets per day.

In [11]:
# Pull in Tweets CSV data.
tweets_otly_df = pd.read_csv(Path("tweetsbyday_otly.csv"))
tweets_otly_df.head()

,Unnamed: 0,date
0,0,2021-08-19
1,1,2021-08-19
2,2,2021-08-19
3,3,2021-08-19
4,4,2021-08-19


In [39]:
# Count tweets per day
date_count = tweets_otly_df["date"]
date_count = Counter(date_count)
date_dict = dict(date_count)
Tweetsperday_OTLY = pd.DataFrame(date_dict.items(), columns = ["Date", "TweetCount"])
Tweetsperday_OTLY

,Date,TweetCount
0,2021-08-19,63
1,2021-08-18,53
2,2021-08-17,91
3,2021-08-16,246
4,2021-08-15,24
...,...,...
87,2021-05-24,57
88,2021-05-23,129
89,2021-05-22,80
90,2021-05-21,274


## Use Alpaca to bring in the OTLY stock data

In [14]:
# Load dotenv
load_dotenv()

True

In [15]:
# Set API keys
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

# Make sure keys have imported correctly.
display(type(ALPACA_API_KEY))
type(ALPACA_SECRET_KEY)

str

str

In [17]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    ALPACA_API_KEY,
    ALPACA_SECRET_KEY,
    api_version="v2")

In [18]:
# Set start and end dates.
start_date = pd.Timestamp("2021-05-20", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-08-20", tz="America/New_York").isoformat()

In [20]:
# Define ticker, time frame, and row limit.
ticker = "OTLY"
timeframe = "1D"
limit_rows =1000

In [21]:
# Get OTLY closing prices for the month after IPO.
otly_df = alpaca.get_barset(
    ticker,
    timeframe,
    start = start_date,
    end = end_date,
    limit = limit_rows
).df

# Display sample data
otly_df.head()

OTLY                               
                            open   high    low  close    volume
time                                                           
2021-05-20 00:00:00-04:00  22.12  22.74  19.99  20.24  42800807
2021-05-21 00:00:00-04:00  22.20  23.25  21.40  22.39  15512845
2021-05-24 00:00:00-04:00  22.61  22.92  20.58  20.71   7285437
2021-05-25 00:00:00-04:00  21.29  21.42  20.11  21.18   4932242
2021-05-26 00:00:00-04:00  21.57  21.87  20.99  21.63   2446139

In [22]:
# Reset OTLY stock data index so date can be cleaned.
otly_df.reset_index(level=0, inplace=True)
otly_df.head()

time   OTLY                               
                              open   high    low  close    volume
0 2021-05-20 00:00:00-04:00  22.12  22.74  19.99  20.24  42800807
1 2021-05-21 00:00:00-04:00  22.20  23.25  21.40  22.39  15512845
2 2021-05-24 00:00:00-04:00  22.61  22.92  20.58  20.71   7285437
3 2021-05-25 00:00:00-04:00  21.29  21.42  20.11  21.18   4932242
4 2021-05-26 00:00:00-04:00  21.57  21.87  20.99  21.63   2446139

In [34]:
# Clean date & setup dataframe with necessary columns
otly_df["time"] = pd.to_datetime(otly_df["time"]).dt.date
otly_df.head()
# Create new dataframe for relevant columns and use for loop to populate it.
otly_df2 = pd.DataFrame()
for column in otly_df:
    otly_df2["time"] = otly_df["time"]
    otly_df2["close"] = otly_df["OTLY"]["close"]
    otly_df2["volume"] = otly_df["OTLY"]["volume"]
otly_df2.tail()

,time,close,volume
60,2021-08-16,16.43,4657371
61,2021-08-17,15.92,6298027
62,2021-08-18,15.16,2714345
63,2021-08-19,15.73,3534728
64,2021-08-20,15.63,2355810


## Use SQL to join dataframes into a single table.

In [38]:
# Create the connection string for your SQLite database
database_connection_string = 'sqlite:///'

# Pass the connection string to the SQLAlchemy create_engine function
engine = sqlalchemy.create_engine(database_connection_string)

# Confirm that the database engine was created.
engine

Engine(sqlite:///)

In [40]:
# Create tweets per day Dataframe in SQL
Tweetsperday_OTLY.to_sql(
    "tweets",
    engine,
    index=False,
    if_exists="replace")
# Create stocks dataframe in SQL
otly_df2.to_sql(
    "stock",
    engine,
    index=False,
    if_exists="replace")

# Check SQL table names
engine.table_names()

/Users/jacob/opt/anaconda3/envs/pandasdev/lib/python3.7/site-packages/ipykernel_launcher.py:15: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  from ipykernel import kernelapp as app


['stock', 'tweets']

In [41]:
# Create JOIN query to join the two tables on each other
query = """
SELECT tweets.Date, tweets.TweetCount, stock.close, stock.volume
FROM stock
JOIN tweets ON tweets.Date = stock.Time
"""

# Run the query
OTLY_joined_df = pd.read_sql_query(query, con=engine)

# Review the resulting Dataframe
OTLY_joined_df

,Date,TweetCount,close,volume
0,2021-08-19,63,15.73,3534728
1,2021-08-18,53,15.16,2714345
2,2021-08-17,91,15.92,6298027
3,2021-08-16,246,16.43,4657371
4,2021-08-13,58,16.85,2704530
...,...,...,...,...
59,2021-05-26,48,21.63,2446139
60,2021-05-25,40,21.18,4932242
61,2021-05-24,57,20.71,7285437
62,2021-05-21,274,22.39,15512845


## Add daily percent change columns to dataframe.

In [42]:
# Sort the dataframe on the date
OTLY_joined_df = OTLY_joined_df.sort_values("Date")
# Add tweet count percent change
OTLY_joined_df["tweet_pct_chg"] = OTLY_joined_df["TweetCount"].pct_change()
# Add close price percent change
OTLY_joined_df["close_pct_chg"] = OTLY_joined_df["close"].pct_change()
# Add volume percent change
OTLY_joined_df["vol_pct_chg"] = OTLY_joined_df["volume"].pct_change()
# Set the date as index on the dataframe
OTLY_joined_df = OTLY_joined_df.set_index("Date")
#Preview the dataframe
OTLY_joined_df

,TweetCount,close,volume,tweet_pct_chg,close_pct_chg,vol_pct_chg
Date,,,,,,
2021-05-20,480,20.24,42800807,NaN,NaN,NaN
2021-05-21,274,22.39,15512845,-0.429167,0.106225,-0.637557
2021-05-24,57,20.71,7285437,-0.791971,-0.075033,-0.530361
2021-05-25,40,21.18,4932242,-0.298246,0.022694,-0.323000
2021-05-26,48,21.63,2446139,0.200000,0.021246,-0.504051
...,...,...,...,...,...,...
2021-08-13,58,16.85,2704530,0.035714,-0.008240,-0.070939
2021-08-16,246,16.43,4657371,3.241379,-0.024926,0.722063
2021-08-17,91,15.92,6298027,-0.630081,-0.031041,0.352271


## Create charts and analysis

In [43]:
# Create pct change overlay chart
tweet_pct_chart = OTLY_joined_df["tweet_pct_chg"].hvplot(rot=90)
close_pct_chart = OTLY_joined_df["close_pct_chg"].hvplot(rot=90)
vol_pct_chart = OTLY_joined_df["vol_pct_chg"].hvplot(rot=90)

overlay = tweet_pct_chart * close_pct_chart * vol_pct_chart
overlay

:Overlay
   .Curve.Tweet_pct_chg :Curve   [Date]   (tweet_pct_chg)
   .Curve.Close_pct_chg :Curve   [Date]   (close_pct_chg)
   .Curve.Vol_pct_chg   :Curve   [Date]   (vol_pct_chg)

In [44]:
# Create correlation table.
OTLY_joined_df.corr()

,TweetCount,close,volume,tweet_pct_chg,close_pct_chg,vol_pct_chg
TweetCount,1.000000,0.025446,0.843256,0.586439,0.130395,0.308849
close,0.025446,1.000000,-0.046213,-0.031758,0.231822,-0.028003
volume,0.843256,-0.046213,1.000000,0.244699,0.138327,0.179161
tweet_pct_chg,0.586439,-0.031758,0.244699,1.000000,-0.041800,0.660993
close_pct_chg,0.130395,0.231822,0.138327,-0.041800,1.000000,-0.100843
vol_pct_chg,0.308849,-0.028003,0.179161,0.660993,-0.100843,1.000000
